Part 3 - Graph Analytics with Cypher
====================================

In this section we will use Neo4j and Bloom to learn Graph Analytics against the Open Sanctions database as outlined on Github at [opensanctions/offshore-graph](https://github.com/opensanctions/offshore-graph#sanctionsoffshores-graph-demo).

The following resources will assist you in learning Cypher to query Neo4j and other platforms via [openCypher]:

* [Neo4j Cypher Manual - Overview](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_overview/)
* [Neo4j Cypher Manual - Cypher and Neo4j](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_neo4j/)
* [Neo4j 5 Cypher Cheat Sheet](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise/)
* [Neo4j Graph Academy Cypher Fundamentals - 1 Hour Video](https://graphacademy.neo4j.com/courses/cypher-fundamentals/)
* [Neo4j Cypher Manual PDF](https://neo4j.com/docs/pdf/neo4j-cypher-manual-5.pdf)
* [Neo4j Getting Started - Query a Neo4j database using Cypher](https://neo4j.com/docs/getting-started/cypher-intro/)
* [Bite-Sized Neo4j for Data Scientists](https://neo4j.com/video/bite-sized-neo4j-for-data-scientists/) [[github code](https://github.com/cj2001/bite_sized_data_science)]
* [Using Neo4j from Python](

## Describe Our Network

Let's start by describing our network...

In [ ]:
MATCH (n) RETURN distinct labels(n), count(*)